Display a dual axis histogram/line chart
- X axis: months
- Y axis bars: monthly publishing counts
- Y Axis line: monthly covid cases



In [1]:
import pandas as pd
import datetime

In [2]:
# read data from files
papersDF = pd.read_csv('data/Papers.csv', header=0)
covidDF = pd.read_csv('data/covid-19-all.csv', header=0)

Aggregate dataframes

Papers
- Month
- Number published in month

Covid
- Month
- Total confirmed cases in month

In [3]:
# format date strings as datetimes
papersDF['Date'] = pd.to_datetime(papersDF['Date'])
papersDF['CreatedDate'] = pd.to_datetime(papersDF['CreatedDate'])
covidDF['Date'] = pd.to_datetime(covidDF['Date'])

In [4]:
# get rid of most recent and future publications
papersDF = papersDF[papersDF['Date'] <= datetime.datetime(2020, 9, 30)].sort_values('Date')
papersDF.tail()


,PaperId,Rank,Date,ReferenceCount,CitationCount,FamilyId,FamilyRank,CreatedDate
65351,3011512697,23024,2020-09-30,15,0,3011512697,22311,2020-03-23
152534,3011220111,23225,2020-09-30,10,0,3011220111,22181,2020-03-23
158494,3016344096,21237,2020-09-30,17,2,3016344096,20931,2020-04-24
146826,3082522724,21370,2020-09-30,46,0,3082522724,20796,2020-09-08
88120,2609568224,22662,2020-09-30,15,0,2609568224,21156,2017-05-05


In [5]:
# get rid of most recent data
covidDF = covidDF[covidDF['Date'] <= datetime.datetime(2020, 9, 30)].sort_values('Date')
covidDF.tail()

,Country/Region,Province/State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
702877,US,Illinois,38.712410,-88.084754,220.0,0.0,6.0,2020-09-30
702878,US,Illinois,41.466138,-90.570422,3071.0,0.0,77.0,2020-09-30
702879,US,Illinois,37.753330,-88.541197,362.0,0.0,4.0,2020-09-30
702881,US,Illinois,40.157180,-90.611142,50.0,0.0,0.0,2020-09-30
705509,Zimbabwe,NaN,-19.015438,29.154857,7838.0,6303.0,228.0,2020-09-30


In [6]:
# set up columns for aggregation
papersAggDF = papersDF[['PaperId', 'Date']]
papersAggDF.head()

,PaperId,Date
118750,1534585743,2018-09-01
126767,2888191606,2018-09-01
126756,2850121008,2018-09-01
61471,1527154779,2018-09-01
61145,2981776382,2018-09-01


In [7]:
# set up columns for aggregation
covidAggDF = covidDF[['Confirmed', 'Date']].fillna(0)
covidAggDF.head()

,Confirmed,Date
0,1.0,2020-01-22
21,1.0,2020-01-22
22,0.0,2020-01-22
23,0.0,2020-01-22
24,2.0,2020-01-22


In [8]:
# aggregate monthly publication count
monthlyPublishTotalsDF = papersAggDF.groupby(pd.Grouper(key='Date', freq='M')) \
    .count() \
    .rename(columns={'PaperId':'Paper Count'})
monthlyPublishTotalsDF['Month'] = monthlyPublishTotalsDF.index.month.astype(str) + '/' + monthlyPublishTotalsDF.index.year.astype(str)
monthlyPublishTotalsDF.head(30)

,Paper Count,Month
Date,,
2018-09-30,5205,9/2018
2018-10-31,4729,10/2018
2018-11-30,3828,11/2018
2018-12-31,4781,12/2018
2019-01-31,13891,1/2019
2019-02-28,4747,2/2019
2019-03-31,5715,3/2019
2019-04-30,5925,4/2019
2019-05-31,6614,5/2019


In [9]:
# monthly case count
monthlyCovidTotalsDF = covidAggDF.groupby(pd.Grouper(key='Date', freq='M')).sum()
monthlyCovidTotalsDF['Month'] = monthlyCovidTotalsDF.index.month.astype(str) + '/' + monthlyCovidTotalsDF.index.year.astype(str)
monthlyCovidTotalsDF.head(10)

,Confirmed,Month
Date,,
2020-01-31,38535.0,1/2020
2020-02-29,1671959.0,2/2020
2020-03-31,8899230.0,3/2020
2020-04-30,63402405.0,4/2020
2020-05-31,144769885.0,5/2020
2020-06-30,246578791.0,6/2020
2020-07-31,431490739.0,7/2020
2020-08-31,671137307.0,8/2020
2020-09-30,893386521.0,9/2020


Use Plottly to create chart
- X axis: months
- Y axis bars: monthly publishing counts
- Y Axis line: monthly covid cases